In [139]:
#word2vec
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import datapath
#import fasttext.util

#utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

#src
#root_project = "/content/SaRaH/"
root_project = "/Users/Alessandro/Dev/repos/SaRaH/"
#root_project = "/home/jupyter/SaRaH/"
sys.path.append(root_project)
from src.data.utils import load_csv_to_dict, dtype, dtype_transformation, set_unkmark_token
from src.features.word_embedding import get_index_key_association, get_int_seq, build_keras_embedding_matrix, get_data_to_emb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dataset

In [3]:
#path
dataset_path    = root_project+'dataset/haspeede2/preprocessed/dev/dev.csv'
w2v_bin_path    = root_project+'results/model/word2vec/twitter128.bin'

In [151]:
dataset = load_csv_to_dict(dataset_path)
dtype(dataset)
print(len(dataset["tokens"]))
print(dataset["tokens"][53])
print(dataset["tokens"][29])

Converting data 'lemma' String in list of String
Converting data 'pos' String in list of String
Converting data 'dep' String in list of String
Converting data 'word_polarity' String in list of String
Converting data 'tokens' String in list of String
Converting data 'stem' String in list of String
id                   <class 'int'>
text                 <class 'str'>
hs                   <class 'int'>
stereotype           <class 'int'>
text_length          <class 'int'>
hashtags             <class 'int'>
%CAPS-LOCK words     <class 'int'>
esclamations         <class 'int'>
questions            <class 'int'>
lemma                <class 'list'>
pos                  <class 'list'>
dep                  <class 'list'>
word_polarity        <class 'list'>
tokens               <class 'list'>
stem                 <class 'list'>
%bad_words           <class 'int'>
400
['+', '+', '+', '<', 'Siria', '>', '🇸', '🇾', 'Evacuati', 'civili', 'dalla', 'città', 'terroristi', '<', 'Is', '>', 'avanzano', 'dopo

In [182]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import emoji
import re

text_processor = TextPreProcessor(
    fix_html=True,  # fix HTML tokens
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Reading english - 1grams ...


In [239]:
print("\u2714")
print("\U00002714")

✔
✔


In [331]:
EMOJI_PATTERN = re.compile(
    "(["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "])"
)
    
def add_space_between_emojies(text):
    text = re.sub(EMOJI_PATTERN, r' \1 ', text)
    return text

def findall_emojies(text):
  # Ref: https://gist.github.com/Alex-Just/e86110836f3f93fe7932290526529cd1#gistcomment-3208085
  # Ref: https://en.wikipedia.org/wiki/Unicode_block
    text = re.findall(EMOJI_PATTERN, text)
    return text

In [476]:
text = "+++#Siria Evacuati civili ✔❤"
text = add_space_between_emojies(text)
decode_text = text.encode('unicode-escape').decode("latin_1")

all_extra_emojii = findall_emojies(text)

print(all_extra_emojii)
extra_emoji = []
for elem in all_extra_emojii:
    extra_emoji.append(elem.encode('unicode-escape').decode("latin_1"))
extra_emoji = {elem : "\\U0000"+elem[2:] for elem in extra_emoji}
extra_emoji

['✔', '❤']


{'\\u2714': '\\U00002714', '\\u2764': '\\U00002764'}

In [360]:
sentence = ' '.join([emoji.UNICODE_EMOJI_ALIAS_ENGLISH[extra_emoji[word]] if word in extra_emoji and extra_emoji[word] in emoji.UNICODE_EMOJI_ALIAS_ENGLISH else word for word in decode_text.split()])
print(sentence)
new_sentence_encoding = (sentence.encode("latin_1")
   .decode("raw_unicode_escape")
   .encode('utf-16', 'surrogatepass')
   .decode('utf-16')
)

+++#Siria Evacuati civili \u2714 \u2764


In [473]:
missing_emoji = {}
for key in emoji.UNICODE_EMOJI_ENGLISH:
    if key not in emoji.UNICODE_EMOJI_ITALIAN:
        missing_emoji[key.encode('unicode-escape').decode("latin_1")] = key
print(len(missing_emoji))
print(len(emoji.UNICODE_EMOJI_ITALIAN))
decoded_unicode_ita = {k.encode('unicode-escape').decode("latin_1"): emoji.UNICODE_EMOJI_ITALIAN[k] for k in emoji.UNICODE_EMOJI_ITALIAN}

missing_emoji_tralation = {}

for elem in missing_emoji:
    for k in decoded_unicode_ita:
        if elem in k:
            missing_emoji_tralation[missing_emoji[elem]] = decoded_unicode_ita[k].replace(':','').replace('_', ' ')
            break

842
3303


In [474]:
len(missing_emoji_tralation)

805

In [486]:
text = "+++#Siria🇸🇾 Evacuati civili ✔❤ 👎👎"
print(text)
text_result = emoji.demojize(text, language='it', delimiters=(" ", " "))
text_result = add_space_between_emojies(text_result)
text_result = text_result.split()
text_result = [elem.replace(':','').replace('_', ' ') if ":"+elem+":" in emoji.EMOJI_UNICODE_ITALIAN else elem for elem in text_result]
text_result = [ missing_emoji_tralation[word] if word in missing_emoji_tralation else word for word in text_result]
' '.join(text_result)
print(' '.join(text_result))

+++#Siria🇸🇾 Evacuati civili ✔❤ 👎👎
+++#Siria bandiera siria Evacuati civili segno di spunta nero cuore rosso pollice verso pollice verso


In [ ]:
### examples data:

In [134]:
s = ['la', 'violenza', 'simbolica', 'può', 'fare', 'molti', 'più', 'danni', 'di', 'qualche', 'auto', 'bruciata', '.', '.', '.', '.', 'i', 'profughi', ',', 'i', 'rom', 'non', 'hanno', "l'", 'assicurazione', '.', '.', '.']
' '.join(map(str, s)).split()
' '.join([str(elem) for elem in s]) 
#print(ast.literal_eval(s))
#' '.join(ast.literal_eval(s)).split()

"la violenza simbolica può fare molti più danni di qualche auto bruciata . . . . i profughi , i rom non hanno l' assicurazione . . ."

In [ ]:
string = ['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'negative', 'neutral', None, 'neutral', 'neutral', 'negative', 'positive', 'neutral', 'neutral', 'neutral']

In [73]:
tokens = ["'","l", "'", "uomo","'","ciao"]
print(tokens)
apostrophes = ["'", "’"]

result_tokens = []
pos = 0
while pos < len(tokens):
    if pos !=(len(tokens)-1) and tokens[pos+1] in apostrophes:
        result_tokens.append(tokens[pos] + "\'")
        pos+=2
    else:
        result_tokens.append(tokens[pos])
        pos+=1
print(result_tokens)  

["'", 'l', "'", 'uomo', "'", 'ciao']
["'", "l'", "uomo'", 'ciao']


In [127]:
result_tokens = ["l'"]
dc = {"prova":[result_tokens]}
df = pd.DataFrame(dc)
print(df)
df.to_csv()

  prova
0  [l']


',prova\n0,"[""l\'""]"\n'

In [114]:
print("[""\'"", ""l\'"", ""uomo\'"", \'ciao\']")

[', l', uomo', 'ciao']


In [90]:
"[""\'"", ""l\'"", ""uomo\'"", \'ciao\']"

b"[', l', uomo', 'ciao']"